# This notebook is the code which will cover the data extraction and manipulation for the capstone

In [1]:
print('importation: begin!')
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

importation: begin!
Libraries imported.


**The cell below extracts data about Italy's provinces, from a wikipedia page. Since some cities are displayed with the english name, we need to use the equivalent italian wikipedia page, which uses approximately the same names as the infected spreadsheet**

In [2]:
url = 'https://it.wikipedia.org/wiki/Province_d%27Italia'

df = pd.read_html(url)

**Since there are more than one data frame in that page, we will need to look into them, to find out which one we're interested in.**

It turns out we care about the fist one, so position 0 in the list of dataframes.

In [3]:
# There are 11 elements in df, which means 11 different dataframes! We care about the fisrt one, position number 0
for i in range(len(df)):
    print('new dataframe\n')
    df_new = df[i]
    print(df_new.head(2))

new dataframe

                     Provincia Sigla   Regione Popolazione(ab.)  \
0  Agrigento (lib. cons. com.)    AG   Sicilia          434 870   
1                  Alessandria    AL  Piemonte          421 284   

  Superficie(km²) Densità(ab./km²) Comuni(N°)          Presidente  \
0        3 052,59              142         43  Roberto Barberi[6]   
1        3 558,83              118        187    Gianfranco Baldi   

                        Partito  
0     Commissario straordinario  
1  Indipendente (Centro-destra)  
new dataframe

   Pos.    Provincia Sigla              Regione  Superficie (km²)
0     1      Sassari    SS             Sardegna           7692090
1     2  Bolzano[21]    BZ  Trentino-Alto Adige           7398381
new dataframe

  V · D · M Suddivisioni dell'Italia  \
0                Regioni statistiche   
1                            Regioni   

                V · D · M Suddivisioni dell'Italia.1  
0  RegioniProvince, province autonome, città metr...  
1  Province, p

In [4]:
# we extract the relevant table by accessing index two of the list
df1 = df[0]
df1.head()

,Provincia,Sigla,Regione,Popolazione(ab.),Superficie(km²),Densità(ab./km²),Comuni(N°),Presidente,Partito
0,Agrigento (lib. cons. com.),AG,Sicilia,434 870,"3 052,59",142,43,Roberto Barberi[6],Commissario straordinario
1,Alessandria,AL,Piemonte,421 284,"3 558,83",118,187,Gianfranco Baldi,Indipendente (Centro-destra)
2,Ancona,AN,Marche,471 228,"1 936,22",240,47,Luigi Cerioni,Indipendente (Centro-sinistra)
3,Aosta[7] (reg.autonoma),AO,Valle d'Aosta,125 666,"3 260,9",39,74,/,/
4,Arezzo,AR,Toscana,342 654,"3 233,08",106,36,Silvia Chiassai,Indipendente (Centro-destra)


**We can drop a few columns, since we're not really interested in many of them**

In [5]:
df2 = df1.drop(axis= 1, columns = ['Sigla', 'Regione', 'Superficie(km²)', 'Comuni(N°)', 'Partito', 'Presidente'])

**We then rename some columns to better fit our needs, using the english equivalent**

In [6]:
df2.rename(columns = {'Provincia':'Province','Popolazione(ab.)':'Population', 'Densità(ab./km²)':'Density'}, inplace = True)

In [7]:
df2.head(15)

,Province,Population,Density
0,Agrigento (lib. cons. com.),434 870,142
1,Alessandria,421 284,118
2,Ancona,471 228,240
3,Aosta[7] (reg.autonoma),125 666,39
4,Arezzo,342 654,106
5,Ascoli Piceno,207 179,169
6,Asti,214 638,142
7,Avellino,418 306,149
8,Bari,1 248 489,326
9,Barletta-Andria-Trani,390 011,253


**As we can see, there is some data cleaning to do on this table. Let's proceed with cleaning up the names. We can see there are many unnecessary brackets information that we need to take out**

In [8]:
regespr = re.compile(r' \(')
regespr1 = re.compile(r'\[')

new_list = list()
for i in df2['Province']:
    #print(i)
    try:
        start_pos = re.search(regespr, i).start()
        new_i = i[:start_pos]
    except:
        start_pos = 0
        new_i = i
    try:
        start_pos1 = re.search(regespr1, new_i).start()
        new_i1 = new_i[:start_pos1]
    except:
        start_pos1 = 0
        new_i1 = new_i
    lower_item = new_i1.lower()
    new_list.append(lower_item)

**There are still some unique records that don't match. We will replace them manually**

In [9]:
print(new_list)
for count,i in enumerate(new_list):
    if i == 'forlì-cesena':
        new_list[count] = 'forlì cesena'
    if i == 'massa-carrara':
        new_list[count] = 'massa carrara' #
    if i == 'monza e brianza':
        new_list[count] = 'monza brianza'
    if i == 'pesaro e urbino':
        new_list[count] = 'pesaro'
print(new_list)

df2['Province adjusted'] = np.array(new_list) # adds column with new list to dataframe
df2.head()

['agrigento', 'alessandria', 'ancona', 'aosta', 'arezzo', 'ascoli piceno', 'asti', 'avellino', 'bari', 'barletta-andria-trani', 'belluno', 'benevento', 'bergamo', 'biella', 'bologna', 'bolzano', 'brescia', 'brindisi', 'cagliari', 'caltanissetta', 'campobasso', 'caserta', 'catania', 'catanzaro', 'chieti', 'como', 'cosenza', 'cremona', 'crotone', 'cuneo', 'enna', 'fermo', 'ferrara', 'firenze', 'foggia', 'forlì-cesena', 'frosinone', 'genova', 'gorizia', 'grosseto', 'imperia', 'isernia', "l'aquila", 'la spezia', 'latina', 'lecce', 'lecco', 'livorno', 'lodi', 'lucca', 'macerata', 'mantova', 'massa-carrara', 'matera', 'messina', 'milano', 'modena', 'monza e brianza', 'napoli', 'novara', 'nuoro', 'oristano', 'palermo', 'padova', 'parma', 'pavia', 'perugia', 'pesaro e urbino', 'pescara', 'piacenza', 'pisa', 'pistoia', 'pordenone', 'potenza', 'prato', 'reggio calabria', 'ragusa', 'ravenna', 'reggio emilia', 'rieti', 'rimini', 'roma', 'rovigo', 'salerno', 'sassari', 'savona', 'siena', 'siracusa'

,Province,Population,Density,Province adjusted
0,Agrigento (lib. cons. com.),434 870,142,agrigento
1,Alessandria,421 284,118,alessandria
2,Ancona,471 228,240,ancona
3,Aosta[7] (reg.autonoma),125 666,39,aosta
4,Arezzo,342 654,106,arezzo


**We will then drop the first province column and then rename the new one to the old one**

In [10]:
df3 = df2.drop(axis= 1, columns = ['Province'])
df3.rename(columns = {'Province adjusted':'Province'}, inplace = True)
df3.head(150)

,Population,Density,Province
0,434 870,142,agrigento
1,421 284,118,alessandria
2,471 228,240,ancona
3,125 666,39,aosta
4,342 654,106,arezzo
5,207 179,169,ascoli piceno
6,214 638,142,asti
7,418 306,149,avellino
8,1 248 489,326,bari
9,390 011,253,barletta-andria-trani


**Now we need to delete those annoying spaces between numbers in the population column**

In [11]:
#regespr2 = re.compile(r' ')

new_list1 = list()
for i in df3['Population']:
    new_list1.append(i)
print(new_list1)
new_list2 = list()
for i in new_list1:
    #print(i)
    #print(len(i))
    if len(i) > 3:
        new_i = i[:-4]+i[-3:]
    else:
        new_i = i
    print(new_i)
    if len(new_i) > 6:
        new_i = new_i[:-7]+new_i[-6:]
    else:
        new_i = new_i
    new_i = int(new_i)
    new_list2.append(new_i)
print(new_list2)

['434\xa0870', '421\xa0284', '471\xa0228', '125\xa0666', '342\xa0654', '207\xa0179', '214\xa0638', '418\xa0306', '1\xa0248\xa0489', '390\xa0011', '202\xa0950', '277\xa0018', '1\xa0114\xa0590', '177\xa0585', '1\xa0017\xa0196', '531\xa0178', '1\xa0265\xa0954', '392\xa0975', '430\xa0372', '262\xa0458', '221\xa0238', '922\xa0965', '1\xa0103\xa0917', '358\xa0316', '385\xa0588', '599\xa0204', '705\xa0753', '358\xa0955', '174\xa0980', '587\xa0098', '164\xa0788', '173\xa0800', '345\xa0691', '1\xa0012\xa0407', '622\xa0183', '394\xa0627', '489\xa0083', '837\xa0427', '139\xa0403', '221\xa0629', '213\xa0840', '84\xa0379', '299\xa0031', '219\xa0556', '575\xa0254', '795\xa0134', '337\xa0380', '334\xa0832', '230\xa0198', '387\xa0876', '314\xa0178', '412\xa0292', '194\xa0878', '197\xa0909', '622\xa0962', '3\xa0261\xa0873', '705\xa0393', '873\xa0935', '3\xa0072\xa0996', '369\xa0018', '208\xa0550', '157\xa0707', '1\xa0245\xa0826', '937\xa0908', '451\xa0631', '545\xa0888', '656\xa0382', '358\xa0886', '31

In [12]:
df3['Population adjusted'] = np.array(new_list2) # adds column with new list to dataframe
df4 = df3.drop(axis= 1, columns = ['Population'])
df4.rename(columns = {'Population adjusted':'Population'}, inplace = True)
df4.head(150)

,Density,Province,Population
0,142,agrigento,434870
1,118,alessandria,421284
2,240,ancona,471228
3,39,aosta,125666
4,106,arezzo,342654
5,169,ascoli piceno,207179
6,142,asti,214638
7,149,avellino,418306
8,326,bari,1248489
9,253,barletta-andria-trani,390011


### Now, we proceed to wrangle the second data source: Since it is a pdf file (http://www.salute.gov.it/imgs/C_17_notizie_4702_1_file.pdf), there are a couple of python packages that allow to wrangle pdf files: tabula and camelot. Of course, none of them is working in this jupiter environment. Even though they get installed with the !pip command, they throw different traceback errors when trying to parse the file. So i had to download it, convert to csv file, and then upload it in my github page. Not ideal, but i have currently no other way to get around the problem.

In [13]:
csv_file = 'https://github.com/EmanueleLanzani/Coursera_Capstone/blob/master/infected_situation_7_may.csv'
inf_df = pd.read_html(csv_file)

In [14]:
inf_df1 = inf_df[0]

In [15]:
inf_df1.head(5)

,Unnamed: 0,Province,Infected
0,NaN,agrigento,135
1,NaN,alessandria,3654
2,NaN,ancona,1822
3,NaN,aosta,1150
4,NaN,arezzo,655


In [16]:
#The first column is clearly a parsing error, so we proceed to drop it:

inf_df2 = inf_df1.drop(axis= 1, columns = ['Unnamed: 0'])
inf_df2.head(150)

,Province,Infected
0,agrigento,135
1,alessandria,3654
2,ancona,1822
3,aosta,1150
4,arezzo,655
5,ascoli piceno,286
6,asti,1655
7,avellino,474
8,bari,1362
9,bat,380


In [18]:
# this will replace the " /" with just a ",", as required by the instructions
inf_df3 = inf_df2.replace('bat','barletta-andria-trani')
inf_df3.head(40)

,Province,Infected
0,agrigento,135
1,alessandria,3654
2,ancona,1822
3,aosta,1150
4,arezzo,655
5,ascoli piceno,286
6,asti,1655
7,avellino,474
8,bari,1362
9,barletta-andria-trani,380


In [19]:
df5 = pd.merge(inf_df3, df4, on='Province', how='inner')

In [20]:
df5.head(150)

,Province,Infected,Density,Population
0,agrigento,135,142,434870
1,alessandria,3654,118,421284
2,ancona,1822,240,471228
3,aosta,1150,39,125666
4,arezzo,655,106,342654
5,ascoli piceno,286,169,207179
6,asti,1655,142,214638
7,avellino,474,149,418306
8,bari,1362,326,1248489
9,barletta-andria-trani,380,253,390011


In [21]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011


In [22]:
from geopy.geocoders import Nominatim 
import geocoder

In [24]:
# define a function to get coordinates
def get_latlng(province):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Italy'.format(province))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [26]:
coords = [ get_latlng(province) for province in df5["Province"].tolist() ]

In [27]:
coords

[[37.31087000000008, 13.576500000000067],
 [44.90724000000006, 8.611560000000054],
 [43.618490000000065, 13.508980000000065],
 [45.73751000000004, 7.320720000000051],
 [43.46354000000008, 11.877650000000074],
 [42.853980000000035, 13.584410000000048],
 [44.90443000000005, 8.199940000000026],
 [40.91217000000006, 14.792880000000025],
 [41.12587000000008, 16.866660000000024],
 [41.17293777700007, 16.171158924000054],
 [46.14098000000007, 12.212750000000028],
 [41.129950000000065, 14.785520000000076],
 [45.69523000000004, 9.66951000000006],
 [45.56041000000005, 8.059780000000046],
 [44.50484000000006, 11.345070000000021],
 [46.49528000000004, 11.353460000000041],
 [45.53689000000003, 10.232000000000028],
 [40.634700000000066, 17.94025000000005],
 [39.214540000000056, 9.110490000000027],
 [37.49004000000008, 14.063220000000058],
 [41.55913000000004, 14.656990000000064],
 [41.07014000000004, 14.331610000000069],
 [37.511360000000025, 15.067520000000059],
 [38.91444000000007, 16.584360000000

In [28]:
len(coords)

108

In [1]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df5['Latitude'] = df_coords['Latitude']
df5['Longitude'] = df_coords['Longitude']
df5.head()

NameError: name 'pd' is not defined

In [1]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: Q1KDITMHCYKL0FVUOCBAYFDNQZNEB3XFPA2O3DIHUWOQBUEF
CLIENT_SECRET:ODJCV2R4CIYTIXPWOTZZXDCTJ4YQ2AFTJSE0USAJVLESLX1S
